In [3]:
import os
import numpy as np
import time
import copy

ang_2_bohr = 1.0/0.52917721067

In [4]:
folder = "/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/"

file_basis_set = folder + "BR"

#file_xyz = folder + "c2h2/p.xyz"
#
#file_molog = folder + "c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog"
#
#file_cp2k_inp = folder + "c2h2/morb_diag_cart/cp2k.inp"

file_xyz = folder + "c2h2/p.xyz"

file_molog = folder + "c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog"

file_cp2k_inp = folder + "c2h2/morb_diag_cart/cp2k.inp"

In [5]:
def spherical_harmonic(l, m, r_vec):
    x = r_vec[:, 0]
    y = r_vec[:, 1]
    z = r_vec[:, 2]
    
    c = 1/(2*np.sqrt(np.pi))
    
    # s orbitals
    if (l, m) == (0, 0):
        return c*np.ones(len(x))
    
    # p orbitals
    elif (l, m) == (1, -1):
        return c*np.sqrt(3)*y
    elif (l, m) == (1, 0):
        return c*np.sqrt(3)*z
    elif (l, m) == (1, 1):
        return c*np.sqrt(3)*x
    
    # d orbitals
    elif (l, m) == (2, -2):
        return c*np.sqrt(15)*x*y
    elif (l, m) == (2, -1):
        return c*np.sqrt(15)*y*z
    elif (l, m) == (2, 0):
        return c*np.sqrt(5)/2*(2*z**2-x**2-y**2)
    elif (l, m) == (2, 1):
        return c*np.sqrt(15)*z*x
    elif (l, m) == (2, 2):
        return c*np.sqrt(15)/2*(x**2-y**2)
    
    print("No spherical harmonic found for l=%d, m=%d" % (l, m))
    return 0

def spherical_harmonic_grid_old(l, m, x_grid, y_grid, z_grid):
    c = 1/(2*np.sqrt(np.pi))
    
    # s orbitals
    if (l, m) == (0, 0):
        return c*np.ones(np.shape(x_grid))
    
    # p orbitals
    elif (l, m) == (1, -1):
        return c*np.sqrt(3)*y_grid
    elif (l, m) == (1, 0):
        return c*np.sqrt(3)*z_grid
    elif (l, m) == (1, 1):
        return c*np.sqrt(3)*x_grid
    
    # d orbitals
    elif (l, m) == (2, -2):
        return c*np.sqrt(15)*x_grid*y_grid
    elif (l, m) == (2, -1):
        return c*np.sqrt(15)*y_grid*z_grid
    elif (l, m) == (2, 0):
        return c*np.sqrt(5)/2*(2*z_grid**2-x_grid**2-y_grid**2)
    elif (l, m) == (2, 1):
        return c*np.sqrt(15)*z_grid*x_grid
    elif (l, m) == (2, 2):
        return c*np.sqrt(15)/2*(x_grid**2-y_grid**2)
    
    print("No spherical harmonic found for l=%d, m=%d" % (l, m))
    return 0

def spherical_harmonic_grid(l, m, x_grid, y_grid, z_grid):
    c = (2.0/np.pi)**(3.0/4.0)
    
    # s orbitals
    if (l, m) == (0, 0):
        return c*np.ones(np.shape(x_grid))
    
    # p orbitals
    elif (l, m) == (1, -1):
        return c*2.0*y_grid
    elif (l, m) == (1, 0):
        return c*2.0*z_grid
    elif (l, m) == (1, 1):
        return c*2.0*x_grid
    
    # d orbitals
    elif (l, m) == (2, -2):
        return c*4.0*x_grid*y_grid
    elif (l, m) == (2, -1):
        return c*4.0*y_grid*z_grid
    elif (l, m) == (2, 0):
        return c*2.0/np.sqrt(3)*(2*z_grid**2-x_grid**2-y_grid**2)
    elif (l, m) == (2, 1):
        return c*4.0*z_grid*x_grid
    elif (l, m) == (2, 2):
        return c*2.0*(x_grid**2-y_grid**2)
    
    print("No spherical harmonic found for l=%d, m=%d" % (l, m))
    return 0

In [6]:
def read_basis_functions(basis_set_file, cp2k_input_file):
    # Find the used basis sets for all used elements
    elem_basis = {}
    with open(cp2k_input_file) as f:
        lines = f.readlines()
        for i in range(len(lines)):
            parts = lines[i].split()
            if parts[0] == "&KIND":
                elem = parts[1]
                for j in range(10):
                    parts = lines[i+j].split()
                    if parts[0] == "BASIS_SET":
                        basis = parts[1]
                        elem_basis[elem] = basis
                        break
    
    basis_sets = {}
    with open(basis_set_file) as f:
        lines = f.readlines()
        for i in range(len(lines)):
            parts = lines[i].split()
            if parts[0] in elem_basis:
                elem = parts[0]
                if parts[1] == elem_basis[elem] or parts[2] == elem_basis[elem]:
                    # We have found the correct basis set
                    print(lines[i], end='')
                    #print(lines[i+1], end='')
                    basis_functions = []
                    nsets = int(lines[i+1])
                    cursor = 2
                    for j in range(nsets):
                        comp = [int(x) for x in lines[i+cursor].split()]
                        #print(comp)
                        n_princ, l_min, l_max, n_exp = comp[:4]
                        l_arr = np.arange(l_min, l_max+1, 1)
                        n_basisf_for_l = comp[4:]
                        assert len(l_arr) == len(n_basisf_for_l)
                        
                        exps = []
                        coeffs = []
                        
                        for k in range(n_exp):
                            exp_c = [float(x) for x in lines[i+cursor+k+1].split()]
                            exps.append(exp_c[0])
                            coeffs.append(exp_c[1:])
                        
                        exps = np.array(exps)
                        coeffs = np.array(coeffs)
                        
                        indx = 0
                        for l, nl in zip(l_arr, n_basisf_for_l):
                            for i in range(nl):
                                #print("l =", l)
                                #print(exps)
                                #print(coeffs[:, indx])
                                basis_functions.append([l, exps, coeffs[:, indx]])
                                indx += 1
                        cursor += n_exp + 1
                        
                    #print()
                    basis_sets[elem] = basis_functions
                    
    return basis_sets

basis_sets = read_basis_functions(file_basis_set, file_cp2k_inp)

 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4


In [7]:
# Secret Magic Normalization
def magic_basis_normalization(basis_sets_):
    basis_sets = copy.deepcopy(basis_sets_)
    for elem, bs in basis_sets.items():
        for shell in bs:
            l = shell[0]
            exps = shell[1]
            coefs = shell[2]
            nexps = len(exps)

            norm_factor = 0
            for i in range(nexps-1):
                for j in range(i+1, nexps):
                    norm_factor += 2*coefs[i]*coefs[j]*(2*np.sqrt(exps[i]*exps[j])/(exps[i]+exps[j]))**((2*l+3)/2)
                    
            #print(elem, shell[0], norm_factor/2)
            for i in range(nexps):
                norm_factor += coefs[i]**2
                
            #print(elem, shell[0], norm_factor)
            #print(elem, shell[0], 1/np.sqrt(norm_factor))

            for i in range(nexps):
                coefs[i] = coefs[i]*exps[i]**((2*l+3)/4)/np.sqrt(norm_factor)
    
    return basis_sets
                
basis_sets_norm = magic_basis_normalization(basis_sets)

In [8]:
# Inspiration from
# https://github.com/ondrejkrejci/PPSTM/blob/master/pyPPSTM/ReadSTM.py#L520
def read_cp2k_MO_file(fn):
    print("Reading CP2K MOs from:"+fn)

    # read all lines into memory
    f = open(fn)
    lines = []
    for l in f.readlines():
        l = l.strip()
        if(len(l)==0): continue
        lines.append(l)
    f.close()
    
    # Remove all but the last section
    delete_line = False
    for i, line in reversed(list(enumerate(lines))):
        if delete_line:
            del lines[i]
        if "MO EIGENVALUES" in line:
            delete_line = True
    
    # detect dimensions
    parts = lines[-3].split()
    nbasis = int(parts[0])
    natoms = int(parts[1])
    nmos = int(lines[-nbasis-5].split()[-1])
    
    
    nlines_per_block = nbasis + 3
    nblocks = int((len(lines)-3)/nlines_per_block)
    
    
    print(nbasis, natoms, nmos, nblocks)
    
    print("Found %d MOs spanned by %d basis functions centered on %d atoms."%(nmos, nbasis, natoms))
    
    assert lines[-1].startswith("HOMO-LUMO gap:")
    assert lines[-2].startswith("Fermi energy:")
    fermi_energy = 27.211385 * float(lines[-2].split()[2])

    # unfold table
    idx = []
    evals = []
    occs = []
    evecs = [list() for i in range(nbasis)]
    labels = [l.split()[:4] for l in lines[4:nbasis+4]]
    
    first_line = 1
    
    for iblock in range(nblocks):
        a = first_line + iblock*nlines_per_block
        evals.extend(lines[a+1].split())
        occs.extend(lines[a+2].split())
        for j in range(nbasis):
            parts = lines[a+3+j].split()
            assert parts[:4] == labels[j]
            evecs[j].extend(parts[4:])

    # convert to numpy arrays
    evals = np.array(evals, float)
    occs = np.array(occs, float)
    evecs = np.array(evecs, float)
    assert evals.shape == (nmos,)
    assert occs.shape == (nmos,)
    assert evecs.shape == (nbasis, nmos)

    # convert hartree to eV
    evals = 27.211385 * evals
    # NB: evecs[basis_idx, morbital_idx] = overlap/projection/scalar_product
    
    #### --------------------------------------------------------------------
    #### Further processing into format
    #### molog_data[atom_nr] = ['H', ['3s', '3px', ...], [[evec for 3s], [evec for 3px], ...]
    molog_data = [[] for i in range(natoms)]
    
    for label, evec in zip(labels, evecs):
        atom_nr = int(label[1]) - 1
        elem = label[2]
        if len(molog_data[atom_nr]) == 0:
            molog_data[atom_nr].extend([elem, [], []])
        molog_data[atom_nr][1].append(label[3])
        molog_data[atom_nr][2].append(evec)
    
    return molog_data, evals, occs, fermi_energy

molog_output = read_cp2k_MO_file(file_molog)
molog_data, evals, occs, fermi_energy = molog_output

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.


In [9]:
# Assuming alphabetical order: x, y, z
# returns in order of increasing m
def cart_coef_to_spherical(l, coefs):
    if l == 0:
        assert len(coefs) == 1
        return np.array(coefs)
    elif l == 1:
        assert len(coefs) == 3
        return np.array([coefs[1], coefs[2], coefs[0]])
    elif l == 2:
        assert len(coefs) == 6
        conv_mat = np.array([[ 0.0, 1.0, 0.0,  0.0, 0.0, 0.0],
                             [ 0.0, 0.0, 0.0,  0.0, 1.0, 0.0],
                             [-0.5, 0.0, 0.0, -0.5, 0.0, 1.0],
                             [ 0.0, 0.0, 1.0,  0.0, 0.0, 0.0],
                             [ 0.5*np.sqrt(3), 0, 0, -0.5*np.sqrt(3), 0, 0]])
        return np.dot(conv_mat, coefs)
    else:
        print("Not implemented.")
        return 0.0

In [10]:
# morb_composition[morb_nr, atom_nr] = [coefs_for_2s, coefs_for_3s, coefs_for_3p, coefs_for_3d, ...]
# coefs_for_3p = [coef_for_m=-1, coef_for_m=0, coef_for_m=1]
def process_molog_output(molog_output):
    
    molog_data, evals, occs, fermi_energy = molog_output
    
    natoms = len(molog_data)
    nmos = len(evals)
    
    morb_composition = [[[] for j in range(natoms)] for i in range(nmos)]

    for i_at in range(natoms):
        elem = molog_data[i_at][0]
        orb_labels = molog_data[i_at][1]
        eig_vecs = molog_data[i_at][2]
        
        i_orb = 0
        while i_orb < len(orb_labels):
            
            n_orb = int(orb_labels[i_orb][0])
            cart_orb = orb_labels[i_orb][1:]
            
            if cart_orb == 's':
                eig_vec = eig_vecs[i_orb]
                for i_mo in range(nmos):
                    morb_composition[i_mo][i_at].append(cart_coef_to_spherical(0, [eig_vec[i_mo]]))
                i_orb += 1
                continue
            
            elif cart_orb == 'px':
                eig_px = eig_vecs[i_orb]
                eig_py = eig_vecs[i_orb+1]
                eig_pz = eig_vecs[i_orb+2]
                for i_mo in range(nmos):
                    spherical_coefs = cart_coef_to_spherical(1, [eig_px[i_mo], eig_py[i_mo], eig_pz[i_mo]])
                    morb_composition[i_mo][i_at].append(spherical_coefs)
                i_orb += 3
                continue
            
            elif cart_orb == 'dx2':
                eig_dx2 = eig_vecs[i_orb]
                eig_dxy = eig_vecs[i_orb+1]
                eig_dxz = eig_vecs[i_orb+2]
                eig_dy2 = eig_vecs[i_orb+3]
                eig_dyz = eig_vecs[i_orb+4]
                eig_dz2 = eig_vecs[i_orb+5]
                for i_mo in range(nmos):
                    spherical_coefs = cart_coef_to_spherical(2, [eig_dx2[i_mo], eig_dxy[i_mo], eig_dxz[i_mo],
                                                                 eig_dy2[i_mo], eig_dyz[i_mo], eig_dz2[i_mo]])
                    morb_composition[i_mo][i_at].append(spherical_coefs)
                i_orb += 6
                continue
            
            else:
                print('Error: found unsupported orbital label')
                break
                
    return morb_composition

morb_composition = process_molog_output(molog_output)

In [11]:
# returns positions in a.u. (Bohr radiuses)
def read_atoms(file_xyz):
    # Read atomic positions (in angstrom)
    data = np.genfromtxt(file_xyz, dtype=None, skip_header=2)
    data = np.atleast_1d(data)
    elems_nrs = []
    positions = []
    for line in data:
        elems_nrs.append([line[0].decode("utf-8"), line[4]])
        positions.append(np.array([line[1], line[2], line[3]]) * ang_2_bohr)
    return positions, elems_nrs

atoms = read_atoms(file_xyz)

In [101]:
def write_cube_file(filename, file_xyz, cell, cell_n, data):
    
    # Read atomic positions (a.u.)
    positions, elems_nrs = read_atoms(file_xyz)
    
    natoms = len(positions)
    origin = np.array([0.0, 0.0, 0.0])
    origin *= ang_2_bohr
    
    f = open(filename, 'w')
    
    f.write('title\n')
    f.write('comment\n')
    
    dv_br = cell/cell_n
    dv_br = dv_br * np.diag([1, 1, 1])
    
    f.write("%5d %12.6f %12.6f %12.6f\n"%(natoms, origin[0], origin[1], origin[2]))
    
    for i in range(3):
        f.write("%5d %12.6f %12.6f %12.6f\n"%(cell_n[i], dv_br[i][0], dv_br[i][1], dv_br[i][2]))
    
    for i in range(natoms):
        at_x = positions[i][0]
        at_y = positions[i][1]
        at_z = positions[i][2]
        f.write("%5d %12.6f %12.6f %12.6f %12.6f\n"%(elems_nrs[i][1], 0.0, at_x, at_y, at_z))
    
    data.tofile(f, sep='\n', format='%12.6e')
    
    f.close()

In [116]:
from scipy.interpolate import RegularGridInterpolator

# Main calculation

# NB: only orthonormal cells are supported

cell = np.array([4.0, 4.0, 7.345])
cell *= ang_2_bohr

#step = 0.08
#step *= ang_2_bohr
#cell_n = np.array((int(cell[0]/step), int(cell[1]/step), int(cell[2]/step)))

cell_n = np.array([50, 50, 90])

def grid_morbital(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n):
    
    ### -------------------------------------------------------------------------------
    ### Load and process data
    load_time = time.time()
    
    # Read atomic positions (a.u.)
    positions, elems_nrs = read_atoms(file_xyz)
    
    # Read MOlog file for molecular orbital decompositon in atomic orbitals
    molog_output = read_cp2k_MO_file(file_molog)
    
    # Process the molog data 
    morb_composition = process_molog_output(molog_output)
    
    # Read basis set info
    basis_sets = read_basis_functions(file_basis_set, file_cp2k_inp)
    basis_sets = magic_basis_normalization(basis_sets)
    
    
    print("----Loading data %.4f s" % (time.time()-load_time))
    ### -------------------------------------------------------------------------------
    ### Set up grids
    grid_time = time.time()
    
    dv = cell/cell_n
    vol_elem = dv[0]*dv[1]*dv[2]
    
    x_arr = np.arange(0, cell_n[0], 1) * dv[0]
    y_arr = np.arange(0, cell_n[1], 1) * dv[1]
    z_arr = np.arange(0, cell_n[2], 1) * dv[2]
    r_grid = np.meshgrid(x_arr, y_arr, z_arr, indexing='ij')
    

    print("----Setting up grids %.4f s" % (time.time()-grid_time))
    ### -------------------------------------------------------------------------------
    ### Calculate all atomic orbitals centered on the midpoint of grid
    aorb_time = time.time()
        
    # Set the origin of the grids to the middle of the cell
    midpoint = cell/2
        
    r_grid_mid = []
    for i in range(3):
        r_grid_mid.append(r_grid[i]-midpoint[i])
    
    r_vec_2 = r_grid_mid[0]**2 + r_grid_mid[1]**2 + r_grid_mid[2]**2
    
    atomic_orbs = {} # {elem}[shell, m] = atomic_orb centered on the grid 
    
    for elem, bs in basis_sets.items():
        atomic_orbs[elem] = []
        for i_shell, shell in enumerate(bs):
            l = shell[0]

            es = shell[1]
            cs = shell[2]

            # Calculate the radial part of the atomic orbital
            radial_part = np.zeros(cell_n)
            for e, c in zip(es, cs):
                radial_part += c*np.exp(-1.0*e*r_vec_2)

            atomic_orbs[elem].append([])
            for i, m in enumerate(range(-l, l+1, 1)):
                atomic_orb = radial_part*spherical_harmonic_grid(l, m,
                                                                 r_grid_mid[0],
                                                                 r_grid_mid[1],
                                                                 r_grid_mid[2])
                atomic_orbs[elem][-1].append(atomic_orb)
                
    print("----Putting atomic orbs on grid %.4f s" % (time.time()-aorb_time))
    ### -------------------------------------------------------------------------------
    ### Set up atomic orb interpolators
    aorb_int_time = time.time()
    
    aorb_interpolators = {} # {elem}[shell, m] = atomic_orb interpolator
    
    for elem, aorbs in atomic_orbs.items():
        aorb_interpolators[elem] = []
        for i_shell, shell in enumerate(aorbs):
            aorb_interpolators[elem].append([])
            for i_m, aorb in enumerate(shell):
                rgi = RegularGridInterpolator((x_arr, y_arr, z_arr), aorb,
                                              bounds_error=False, fill_value=0.0)
                aorb_interpolators[elem][-1].append(rgi)
                
    print("----Setting up interpolators %.4f s" % (time.time()-aorb_int_time))
    ### -------------------------------------------------------------------------------
    ### Calculate molecular orbitals on the grid
    morb_time = time.time()
    
    # one data set on the grid for every molecular orbital
    morbs = [np.zeros(cell_n) for _ in range(len(morb_composition))]
    
    for i_at in range(len(positions)):
        elem = elems_nrs[i_at][0]
        pos = positions[i_at]
        
        for i_shell, shell in enumerate(atomic_orbs[elem]):
            for i_m, at_orb in enumerate(shell):
                
                # Shift the atomic orbital on the grid to right position
                # Also take periodic boundary conditions into account
                # Note: To be accurate, the values in the new position
                # need to be interpolated from the original orbital
                shifted_at_orb = np.copy(at_orb)
                shift_vec = pos - midpoint
                
                num_prim = shift_vec/dv
                roll_vec = np.round(num_prim).astype(int)
                interp_vec = (num_prim - np.round(num_prim))*dv
                
                print(shift_vec, num_prim, interp_vec)
                
                # Procedure: 
                # 1) Interpolate the original orbital in slightly
                #    shifted lattice points that match the destination
                # 2) Roll the orbital to correct position (considering PBC)
                
                interp_f = aorb_interpolators[elem][i_shell][i_m]
                
                interp_grid = np.stack([r_grid[0].flatten(), r_grid[1].flatten(), r_grid[2].flatten()], axis=1)
                interp_grid[:, 0] -= interp_vec[0]
                interp_grid[:, 1] -= interp_vec[1]
                interp_grid[:, 2] -= interp_vec[2]
                
                shifted_at_orb = interp_f(interp_grid).reshape(cell_n)
                
                for i in range(3):
                    shifted_at_orb = np.roll(shifted_at_orb, roll_vec[i], axis=i)
                
                #if i_at == 0:
                #    fname1 = "ao-mid%d_%d_%d.cube" % (i_at, i_shell, i_m)
                #    write_cube_file(fname1, file_xyz, cell, cell_n, at_orb)
                #    fname2 = "ao-shift%d_%d_%d.cube" % (i_at, i_shell, i_m)
                #    write_cube_file(fname2, file_xyz, cell, cell_n, shifted_at_orb)
                
                # Add the atomic orbital to each molecular orbital with
                # the corresponding coefficient
                for i_mo in range(len(morb_composition)):
                    coef = morb_composition[i_mo][i_at][i_shell][i_m]
                    morbs[i_mo] += coef*shifted_at_orb
                
    print("----Putting molecular orbs on grid %.4f s" % (time.time()-morb_time))
    return morbs, atomic_orbs


In [117]:
morbs_on_grid, aorbs_on_grid = grid_morbital(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4
----Loading data 0.2657 s
----Setting up grids 0.0041 s
----Putting atomic orbs on grid 0.2374 s
----Setting up interpolators 0.0008 s
[ 0.          0.         -3.16056694] [  0.           0.         -20.49353302] [ 0.          0.         -0.07611397]
[ 0.          0.         -3.16056694] [  0.           0.         -20.49353302] [ 0.          0.         -0.07611397]
[ 0.          0.         -3.16056694] [  0.           0.         -20.49353302] [ 0.          0.         -0.07611397]
[ 0.          0.         -3.16056694] [  0.           0.         -20.49353302] [ 0.          0.         -0.07611397]
[ 0.          0.         -3.16056694] [  0.           0.         -20.49353302] [ 0.          0.         -0.07611397]


In [115]:
dv = cell/cell_n
vol_elem = dv[0]*dv[1]*dv[2]

# Calculate overlap between two orbitals
# vol_elem in bohr!
def overlap(data1, data2, vol_elem):
    return np.abs(np.dot(data1.flatten(), data2.flatten())*vol_elem)

print("Molecular orbital overlap matrix")
for i in range(len(morbs_on_grid)):
    print("%10d " %i, end='')
print()
for i in range(len(morbs_on_grid)):
    for j in range(len(morbs_on_grid)):
        ol = overlap(morbs_on_grid[i], morbs_on_grid[j], vol_elem)
        print("%10.4f " % ol, end='')
    print()
    

Molecular orbital overlap matrix
         0          1          2          3          4          5          6          7          8          9 
    0.9875     0.0000     0.0042     0.0002     0.0003     0.0000     0.0000     0.0458     0.0003     0.0002 
    0.0000     0.9822     0.0000     0.0000     0.0000     0.0003     0.0004     0.0006     0.0045     0.0545 
    0.0042     0.0000     0.9822     0.0001     0.0001     0.0000     0.0000     0.0085     0.0005     0.0005 
    0.0002     0.0000     0.0001     1.0520     0.0000     0.0000     0.0000     0.0007     0.0000     0.0000 
    0.0003     0.0000     0.0001     0.0000     1.0520     0.0000     0.0000     0.0010     0.0000     0.0000 
    0.0000     0.0003     0.0000     0.0000     0.0000     1.0610     0.0000     0.0000     0.0001     0.0015 
    0.0000     0.0004     0.0000     0.0000     0.0000     0.0000     1.0610     0.0000     0.0001     0.0019 
    0.0458     0.0006     0.0085     0.0007     0.0010     0.0000     0.0000   

In [ ]:
print()
print("Atomic orbital overlap matrix")
for i_at in range(len(aorbs_on_grid)):
    for i_nr in range(len(aorbs_on_grid[i_at])):
        for i_m in range(len(aorbs_on_grid[i_at][i_nr])):
            print(" (%d %d %d)" %(i_at, i_nr, i_m), end='')
print()
for i_at in range(len(aorbs_on_grid)):
    for i_nr in range(len(aorbs_on_grid[i_at])):
        for i_m in range(len(aorbs_on_grid[i_at][i_nr])):
            for j_at in range(len(aorbs_on_grid)):
                for j_nr in range(len(aorbs_on_grid[j_at])):
                    for j_m in range(len(aorbs_on_grid[j_at][j_nr])):
                        ol = overlap(aorbs_on_grid[i_at][i_nr][i_m], aorbs_on_grid[j_at][j_nr][j_m], vol_elem)
                        print("%7.3f " % ol, end='')
            print()

In [17]:
for i in range(len(morbs_on_grid)):
    fname = "mo%d.cube" % i
    write_cube_file(fname, file_xyz, cell, cell_n, morbs_on_grid[i])

In [18]:
for i_at in range(len(aorbs_on_grid)):
    for i_nr in range(len(aorbs_on_grid[i_at])):
        for i_m in range(len(aorbs_on_grid[i_at][i_nr])):
            fname = "ao%d_%d_%d.cube" % (i_at, i_nr, i_m)
            write_cube_file(fname, file_xyz, cell, cell_n, aorbs_on_grid[i_at][i_nr][i_m])

In [134]:
# Main calculation

#cell = np.array([[12.0, 0.0, 0.0], [0.0, 12.0, 0.0], [0.0, 0.0, 12.0]])
#cell *= ang_2_bohr

#step = 0.08
#step *= ang_2_bohr
#cell_n = np.array((int(cell[0, 0]/step), int(cell[1, 1]/step), int(cell[2, 2]/step)))

#cell_n = np.array([150, 150, 150])

def grid_morbital(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n):
    
    ### -------------------------------------------------------------------------------
    ### Load and process data
    load_time = time.time()
    
    # Read atomic positions (a.u.)
    positions, elems_nrs = read_atoms(file_xyz)
    
    # Read MOlog file for molecular orbital decompositon in atomic orbitals
    molog_output = read_cp2k_MO_file(file_molog)
    
    # Process the molog data 
    morb_composition = process_molog_output(molog_output)
    
    # Read basis set info
    basis_sets = read_basis_functions(file_basis_set, file_cp2k_inp)
    basis_sets = magic_basis_normalization(basis_sets)
    
    
    print("----Loading data %.4f s" % (time.time()-load_time))
    ### -------------------------------------------------------------------------------
    ### Set up grids
    grid_time = time.time()
    
    dv = cell/cell_n
    vol_elem = np.linalg.det(dv)
    
    # Slow version
    #x_grid = np.zeros(cell_n)
    #y_grid = np.zeros(cell_n)
    #z_grid = np.zeros(cell_n)
    #for xi in range(cell_n[0]):
    #    for yi in range(cell_n[1]):
    #        for zi in range(cell_n[2]):
    #            (x, y ,z) = xi*dv[0] + yi*dv[1] + zi*dv[2]
    #            x_grid[xi, yi, zi] = x
    #            y_grid[xi, yi, zi] = y
    #            z_grid[xi, yi, zi] = z
    #r_grid = [x_grid, y_grid, z_grid]
    
    xi_grid = np.arange(0, cell_n[0], 1)
    yi_grid = np.arange(0, cell_n[1], 1)
    zi_grid = np.arange(0, cell_n[2], 1)
    xi_grid, yi_grid, zi_grid = np.meshgrid(xi_grid, yi_grid, zi_grid, indexing='ij')
    r_grid = []
    for i in range(3):
        r_grid.append(np.zeros(cell_n))
        r_grid[-1] += xi_grid * dv[0, i]
        r_grid[-1] += yi_grid * dv[1, i]
        r_grid[-1] += zi_grid * dv[2, i]

    print("----Setting up grids %.4f s" % (time.time()-grid_time))
    ### -------------------------------------------------------------------------------
    ### Calculate atomic and molecular orbitals on grid
    morb_time = time.time()
    
    # one data set on the grid for every molecular orbital
    morbs = [np.zeros(cell_n) for _ in range(len(morb_composition))]
    
    aorbs = []
    
    for i_at in range(len(positions)):
        elem = elems_nrs[i_at][0]
        pos = positions[i_at]
        
        x_grid_rel = r_grid[0] - pos[0]
        y_grid_rel = r_grid[1] - pos[1]
        z_grid_rel = r_grid[2] - pos[2]
        
        r_vec_2 = x_grid_rel**2 + y_grid_rel**2 + z_grid_rel**2
        
        aorbs.append([])
            
        for i_bs, bs in enumerate(basis_sets[elem]):
            l = bs[0]

            es = bs[1]
            cs = bs[2]
            
            # Calculate the radial part of the atomic orbital
            radial_part = np.zeros(cell_n)
            for e, c in zip(es, cs):
                radial_part += c*np.exp(-1.0*e*r_vec_2)
            
            aorbs[-1].append([])
            
            for i, m in enumerate(range(-l, l+1, 1)):
                atomic_orb = radial_part*spherical_harmonic_grid(l, m, x_grid_rel,
                                                                 y_grid_rel, z_grid_rel)                        
                aorbs[-1][-1].append(atomic_orb)
                if i_at == 0:
                    fname1 = "ao-real%d_%d_%d.cube" % (i_at, i_bs, i)
                    write_cube_file(fname1, file_xyz, cell, cell_n, atomic_orb)
                
                # Add the atomic orbital with the correct coefficient to the molecular orbitals
                for i_mo in range(len(morb_composition)):
                    coef = morb_composition[i_mo][i_at][i_bs][i]
                    morbs[i_mo] += coef*atomic_orb
                
    print("----Main calc %.4f s" % (time.time()-morb_time))    
    return morbs, aorbs

In [135]:
morbs_on_grid, aorbs_on_grid = grid_morbital(file_xyz, file_molog, file_basis_set, file_cp2k_inp, cell, cell_n)

Reading CP2K MOs from:/home/kristjan/sshfs/daint_scratch/cp2k_cnt_orbitals/c2h2/morb_diag_cart/PROJ-COEFF-1_0.MOLog
38 4 10 5
Found 10 MOs spanned by 38 basis functions centered on 4 atoms.
 H  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q1
 C  DZVP-MOLOPT-SR-GTH DZVP-MOLOPT-SR-GTH-q4
----Loading data 0.3760 s
----Setting up grids 0.0087 s
----Main calc 4.3181 s
